REPASO - SESION 5

WITH Y FUNCIONES ANALÍTICAS (OVER Y PARTITION BY)

# Ejercicio 2. Outliers

**Detecta los outliers en la tabla DATOS\_BASE e impútales el valor medio calculado sin outliers**

1. Calcular la media y desviación típica de los ingresos medios por tonelada
2. Marcar como outliers aquellos registros que están a más de dos desviaciones típicas de la media (según la distribución que tengamos puede ser habitual utilizar otro número de desviaciones).
3. Calcular la media sin los outliers.
4. Sustituir el valor original de los outliers por la media sin outliers.

Proceso:

1. Obtén, de DATOS\_BASE, una versión simplificada que contenga la media de euros por tonelada a nivel producto ¿crees que esta granularidad es suficiente para tener un buen resultado?
2. Obtén la media y desviación típica de INGRESOS\_EUR, 
3. Incluye la consulta anterior en un WITH y construye una consulta que detecte los outliers situados a más de 2 desviaciones típicas de la media, prueba también con 1 y 3 desviaciones típicas, ¿cuál tiene más sentido para encontrar un valor atípico?
4. Incluye las dos consultas anteriores en dos etapas de WITH (1º media y desviación, 2º marcar como atípico) y calcula la media de INGRESOS\_EUR con la granularidad adecuada sin tener en cuenta los outliers.
5. Combina todo lo anterior en tres pasos de WITH para sustituir los valores marcados como outlier por la media sin outliers.

In [2]:
-- Paso 2
WITH PRECIOS as (
    SELECT PEDIDO_ID, PRODUCTO_ID, INGRESOS_EUR/TONELADAS AS EUR_TON
    FROM NEMINA.DBO.DATOS_BASE
)
SELECT 
    PRODUCTO_ID, 
    AVG(EUR_TON) AS PROMEDIO, 
    STDEV(EUR_TON) AS DESVIACION
FROM PRECIOS
GROUP BY PRODUCTO_ID

(3 rows affected)

Total execution time: 00:00:00.090

PRODUCTO_ID,PROMEDIO,DESVIACION
Algas,612.29790921195736095521,14.104478109643264
Belleza,3266.10812665621593433937,151.5269507126278
Setas,723.22113376651059018557,47.03893532931351


In [13]:
WITH PRECIOS AS (
    SELECT PEDIDO_ID, PRODUCTO_ID, INGRESOS_EUR/TONELADAS AS EUR_TON, INGRESOS_EUR
    FROM NEMINA.DBO.DATOS_BASE

), PROMEDIOS AS (
    SELECT 
        PRODUCTO_ID, 
        AVG(EUR_TON) AS PROMEDIO, 
        STDEV(EUR_TON) AS DESVIACION
    FROM PRECIOS
    GROUP BY PRODUCTO_ID
)

SELECT P.PEDIDO_ID, 
    P.PRODUCTO_ID, 
    P.EUR_TON, 
    P.INGRESOS_EUR,
    PR.PROMEDIO, 
    PR.DESVIACION,
    (PR.PROMEDIO - 2*PR.DESVIACION) AS EXTMIN,
    (PR.PROMEDIO + 2*PR.DESVIACION) AS EXTMAX,
    CASE WHEN (P.EUR_TON BETWEEN (PR.PROMEDIO - 2*PR.DESVIACION) AND (PR.PROMEDIO + 2*PR.DESVIACION)) THEN 0 ELSE 1 END AS ES_OUTLIER
FROM PRECIOS P
LEFT JOIN PROMEDIOS PR
       ON P.PRODUCTO_ID = PR.PRODUCTO_ID

(501 rows affected)

Total execution time: 00:00:00.336

PEDIDO_ID,PRODUCTO_ID,EUR_TON,INGRESOS_EUR,PROMEDIO,DESVIACION,EXTMIN,EXTMAX,ES_OUTLIER
P-0001,Setas,776.00628930817610062893,24677.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0002,Setas,784.00000000000000000000,27048.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0003,Setas,720.00000000000000000000,27432.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0004,Setas,728.01120448179271708683,25990.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0005,Setas,751.98776758409785932721,24590.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0006,Setas,752.01149425287356321839,26170.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0007,Setas,768.00000000000000000000,25344.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0008,Setas,744.01234567901234567901,24106.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0009,Setas,735.98765432098765432098,23846.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0
P-0010,Setas,727.99283154121863799283,20311.0000000000,723.22113376651059018557,47.03893532931351,629.1432631078835,817.2990044251376,0


In [2]:
WITH PRECIOS AS (
    SELECT PEDIDO_ID, PRODUCTO_ID, INGRESOS_EUR/TONELADAS AS EUR_TON, INGRESOS_EUR
    FROM NEMINA.DBO.DATOS_BASE

), PROMEDIOS AS (
    SELECT 
        PRODUCTO_ID, 
        AVG(EUR_TON) AS PROMEDIO, 
        STDEV(EUR_TON) AS DESVIACION
    FROM PRECIOS
    GROUP BY PRODUCTO_ID
),

DETECTADOS AS (
    SELECT P.PEDIDO_ID, 
        P.PRODUCTO_ID, 
        P.EUR_TON, 
        P.INGRESOS_EUR,
        PR.PROMEDIO, 
        PR.DESVIACION,
        CASE WHEN (P.EUR_TON BETWEEN (PR.PROMEDIO - 2*PR.DESVIACION) AND (PR.PROMEDIO + 2*PR.DESVIACION)) THEN 0 ELSE 1 END AS ES_OUTLIER
    FROM PRECIOS P
    LEFT JOIN PROMEDIOS PR
        ON P.PRODUCTO_ID = PR.PRODUCTO_ID
),

ARMONIZADO AS (
    SELECT DISTINCT PRODUCTO_ID, 
                    AVG(EUR_TON) OVER (PARTITION BY PRODUCTO_ID) AS EUR_TON_H,
                    AVG(INGRESOS_EUR) OVER (PARTITION BY PRODUCTO_ID) AS INGRESOS_EUR_H
    FROM DETECTADOS
    WHERE ES_OUTLIER = 0
)

SELECT D.*,
       CASE WHEN ES_OUTLIER = 0 THEN D.EUR_TON ELSE A.EUR_TON_H END AS EUR_TON_CORREGIDO,
       CASE WHEN ES_OUTLIER = 0 THEN D.INGRESOS_EUR ELSE A.INGRESOS_EUR_H END AS INGRESOS_EUR_CORREGIDO
FROM DETECTADOS D
LEFT JOIN ARMONIZADO A 
    ON D.PRODUCTO_ID = A.PRODUCTO_ID;

(501 rows affected)

Total execution time: 00:00:00.510

PEDIDO_ID,PRODUCTO_ID,EUR_TON,INGRESOS_EUR,PROMEDIO,DESVIACION,ES_OUTLIER,EUR_TON_CORREGIDO,INGRESOS_EUR_CORREGIDO
P-0001,Setas,776.00628930817610062893,24677.0000000000,723.22113376651059018557,47.03893532931351,0,776.00628930817610062893,24677.0000000000
P-0002,Setas,784.00000000000000000000,27048.0000000000,723.22113376651059018557,47.03893532931351,0,784.00000000000000000000,27048.0000000000
P-0003,Setas,720.00000000000000000000,27432.0000000000,723.22113376651059018557,47.03893532931351,0,720.00000000000000000000,27432.0000000000
P-0004,Setas,728.01120448179271708683,25990.0000000000,723.22113376651059018557,47.03893532931351,0,728.01120448179271708683,25990.0000000000
P-0005,Setas,751.98776758409785932721,24590.0000000000,723.22113376651059018557,47.03893532931351,0,751.98776758409785932721,24590.0000000000
P-0006,Setas,752.01149425287356321839,26170.0000000000,723.22113376651059018557,47.03893532931351,0,752.01149425287356321839,26170.0000000000
P-0007,Setas,768.00000000000000000000,25344.0000000000,723.22113376651059018557,47.03893532931351,0,768.00000000000000000000,25344.0000000000
P-0008,Setas,744.01234567901234567901,24106.0000000000,723.22113376651059018557,47.03893532931351,0,744.01234567901234567901,24106.0000000000
P-0009,Setas,735.98765432098765432098,23846.0000000000,723.22113376651059018557,47.03893532931351,0,735.98765432098765432098,23846.0000000000
P-0010,Setas,727.99283154121863799283,20311.0000000000,723.22113376651059018557,47.03893532931351,0,727.99283154121863799283,20311.0000000000


FUNCIONES ANALÍTICAS COMPLEJAS

El número de funciones analíticas es elevado respecto al número de agregados.

- SUM -\> Suma
- AVG -\> Promedio
- STDEV -\> Desviación estándar
- MAX -\> Máximo
- MIN -\> Mínimo
- COUNT -\> Conteo
- LAG -\> Devuelve un valor anterior al actual
- LEAD -\> Devuelve un valor posterior al actual
- FIRST\_VALUE -\> Devuelve el primer valor de una partición
- LAST\_VALUE -\> Devuelve el último valor de una partición
- CUME\_DIST -\> Porcentaje de valores que, en la distribución actual, son inferiores o iguales al actual
- PERCENTILE -\> Devuelve el percentile en el que estamos si tuviésemos una distribución ordenada por un valor

Además de tener un número de funciones analíticas elevado, el número de opciones que pueden ir en el OVER genera muchas combinaciones:

- OVER () Indica que nos referimos al total de los datos
- OVER (PARTITION BY XXX, YYY) Queremos que nos calcule la función dividiendo los datos en grupos que tengan en común las columnas XXX e YYY. Muy habitual para hacer sumas por Año, producto,...
- OVER (PARTITION BY XXX, YYY ORDER BY ZZZ) para las funciones como LAG, LEAD, CUME\_DIST, PERCENTILE en las que importa el orden, este predicado nos permite indicar cómo realizar la ordenación (es obligatorio en las funciones que requieren orden).
- OVER (PARTITION BY XXX, YYY ORDER BY ZZZ ROWS BETWEEN nn PRECEDING AND nn FOLLOWING) permite limitar el número de filas en las que se aplica el agregado, utilizando, por ejemplo, en medias móviles.

In [3]:
-- Ejemplo de LAG, LEAD, FIRST_VALUE
SELECT top 5 -- Sacamos solo 5 para poder convertir a PDF
    PEDIDO_ID,
    PRODUCTO_ID,
    LAG(PEDIDO_ID) OVER(ORDER BY PEDIDO_ID) AS PEDIDO_ANTERIOR,
    LEAD(PEDIDO_ID) OVER (ORDER BY PEDIDO_ID) AS PEDIDO_SIGUIENTE,
    FIRST_VALUE(PEDIDO_ID) OVER (PARTITION BY PRODUCTO_ID ORDER BY PEDIDO_ID) AS PRIMER_PEDIDO_MISMO_PRODUCTO,
    LAST_VALUE(PEDIDO_ID) OVER (PARTITION BY PRODUCTO_ID ORDER BY PEDIDO_ID  ROWS BETWEEN 1 FOLLOWING AND UNBOUNDED FOLLOWING ) AS ULTIMO_PEDIDO_MISMO_PRODUCTO,
    INGRESOS_EUR,
    AVG(INGRESOS_EUR) OVER (ORDER BY PEDIDO_ID ROWS BETWEEN 3 PRECEDING AND 0 FOLLOWING) AS MEDIA_MOVIL_3_PEDIDOS,
    AVG(INGRESOS_EUR) OVER (PARTITION BY PRODUCTO_ID ORDER BY PEDIDO_ID ROWS BETWEEN 3 PRECEDING AND 0 FOLLOWING) AS MEDIA_MOVIL_3_PEDIDOS_PROD
FROM NEMINA.DBO.DATOS_BASE
ORDER BY PEDIDO_ID

(5 rows affected)

Total execution time: 00:00:00.154

PEDIDO_ID,PRODUCTO_ID,PEDIDO_ANTERIOR,PEDIDO_SIGUIENTE,PRIMER_PEDIDO_MISMO_PRODUCTO,ULTIMO_PEDIDO_MISMO_PRODUCTO,INGRESOS_EUR,MEDIA_MOVIL_3_PEDIDOS,MEDIA_MOVIL_3_PEDIDOS_PROD
P-0001,Setas,NULL,P-0002,P-0001,P-0500,24677.0000000000,24677.0000000000,24677.0000000000
P-0002,Setas,P-0001,P-0003,P-0001,P-0500,27048.0000000000,25862.5000000000,25862.5000000000
P-0003,Setas,P-0002,P-0004,P-0001,P-0500,27432.0000000000,26385.6666666666,26385.6666666666
P-0004,Setas,P-0003,P-0005,P-0001,P-0500,25990.0000000000,26286.7500000000,26286.7500000000
P-0005,Setas,P-0004,P-0006,P-0001,P-0500,24590.0000000000,26265.0000000000,26265.0000000000


NTB EJERCICIOS (HOJA 6)

# 1\. Pedidos más altos por producto

# 

Obtén, para cada producto, el mayor número de toneladas compradas.

In [7]:
SELECT DISTINCT PRODUCTO_ID, MAX(TONELADAS) OVER (PARTITION BY PRODUCTO_ID) AS MAX_TN
FROM NEMINA.DBO.DATOS_BASE

(3 rows affected)

Total execution time: 00:00:00.137

PRODUCTO_ID,MAX_TN
Algas,259.0000000000
Belleza,46.0000000000
Setas,592.0000000000


# 2\. Mayores compradores por producto

# 

Obtén el ranking de los 10 clientes que más toneladas han comprado por producto

In [37]:
WITH TONELADAS_PROD AS (
    SELECT DISTINCT CLIENTE_ID,
                PRODUCTO_ID,
                SUM(TONELADAS) OVER (PARTITION BY PRODUCTO_ID,CLIENTE_ID) AS TONELADAS
    FROM NEMINA.DBO.DATOS_BASE
),

RANKED AS (
    SELECT *,ROW_NUMBER() OVER (PARTITION BY PRODUCTO_ID ORDER BY TONELADAS DESC) AS RN FROM TONELADAS_PROD
)

SELECT * FROM RANKED
WHERE RN<=10

(30 rows affected)

Total execution time: 00:00:00.079

CLIENTE_ID,PRODUCTO_ID,TONELADAS,RN
22,Algas,515.5000000000,1
27,Algas,515.2000000000,2
4,Algas,496.6000000000,3
23,Algas,490.0000000000,4
25,Algas,482.1000000000,5
17,Algas,465.1000000000,6
7,Algas,451.7000000000,7
18,Algas,369.0000000000,8
26,Algas,356.0000000000,9
13,Algas,343.6000000000,10


# 3\. Pedidos por encima de la media de toneladas por producto

Obtén, para cada producto, los pedidos cuyas toneladas superen la media de las toneladas vendidas por producto

In [46]:
WITH TONELADAS_PRODUCTO AS (
    SELECT DISTINCT PRODUCTO_ID, AVG(TONELADAS) OVER (PARTITION BY PRODUCTO_ID) AS TN_MEDIA
    FROM NEMINA.DBO.DATOS_BASE
)

SELECT D.PEDIDO_ID, D.PRODUCTO_ID,D.TONELADAS, T.TN_MEDIA
FROM NEMINA.DBO.DATOS_BASE D
LEFT JOIN TONELADAS_PRODUCTO T
    ON T.PRODUCTO_ID=D.PRODUCTO_ID
WHERE D.TONELADAS>T.TN_MEDIA

(104 rows affected)

Total execution time: 00:00:00.098

PEDIDO_ID,PRODUCTO_ID,TONELADAS,TN_MEDIA
P-0051,Algas,43.8000000000,42.9914893617
P-0054,Belleza,44.1000000000,40.7385964912
P-0059,Algas,44.1000000000,42.9914893617
P-0069,Algas,44.1000000000,42.9914893617
P-0075,Algas,44.8000000000,42.9914893617
P-0088,Belleza,43.4000000000,40.7385964912
P-0126,Belleza,44.5000000000,40.7385964912
P-0135,Belleza,42.0000000000,40.7385964912
P-0141,Algas,43.1000000000,42.9914893617
P-0162,Setas,180.0000000000,47.8136718750


# 4\. Productos más vendidos por zona

Obtén, para cada zona, los producto ordenados del más vendido al menos vendido.

In [48]:
SELECT DISTINCT ZONA_ID,PRODUCTO_ID, SUM(TONELADAS) OVER (PARTITION BY ZONA_ID,PRODUCTO_ID) AS TOTAL
FROM NEMINA.DBO.DATOS_BASE D
ORDER BY ZONA_ID ASC,TOTAL DESC

(12 rows affected)

Total execution time: 00:00:00.137

ZONA_ID,PRODUCTO_ID,TOTAL
Coruña,Algas,3949.2000000000
Coruña,Setas,2901.5000000000
Coruña,Belleza,1043.9000000000
Lugo,Algas,1283.6000000000
Lugo,Setas,801.1000000000
Lugo,Belleza,211.7000000000
Ourense,Setas,2746.8000000000
Ourense,Algas,762.0000000000
Ourense,Belleza,251.6000000000
Pontevedra,Setas,5790.9000000000


# 5\. Sólo el mejor producto por zona

Modifica el comando anterior para que devuelva, para cada zona únicamente el producto más vendido. Utiliza la función analítica `ROW_NUMBER`

In [53]:
WITH TOTAL_ZONA AS (
    SELECT DISTINCT ZONA_ID,PRODUCTO_ID, SUM(TONELADAS) OVER (PARTITION BY ZONA_ID,PRODUCTO_ID) AS TOTAL
    FROM NEMINA.DBO.DATOS_BASE
),

RANKED AS (
    SELECT *,ROW_NUMBER() OVER(PARTITION BY ZONA_ID ORDER BY ZONA_ID ASC,TOTAL DESC) AS RN 
    FROM TOTAL_ZONA
)

SELECT * FROM RANKED WHERE RN=1

(4 rows affected)

Total execution time: 00:00:00.071

ZONA_ID,PRODUCTO_ID,TOTAL,RN
Coruña,Algas,3949.2000000000,1
Lugo,Algas,1283.6000000000,1
Ourense,Setas,2746.8000000000,1
Pontevedra,Setas,5790.9000000000,1


# 6\. Obtén, para cada zona, todos los pedidos del producto más vendido

In [57]:
WITH TOTAL_ZONA AS (
    SELECT DISTINCT ZONA_ID,PRODUCTO_ID, SUM(TONELADAS) OVER (PARTITION BY ZONA_ID,PRODUCTO_ID) AS TOTAL
    FROM NEMINA.DBO.DATOS_BASE
),

RANKED AS (
    SELECT *,ROW_NUMBER() OVER(PARTITION BY ZONA_ID ORDER BY ZONA_ID ASC,TOTAL DESC) AS RN 
    FROM TOTAL_ZONA
),

PRODUCTO_ZONA AS (
    SELECT * FROM RANKED WHERE RN=1
)

SELECT DISTINCT D.PEDIDO_ID, D.PRODUCTO_ID, D.ZONA_ID, P.RN
FROM NEMINA.DBO.DATOS_BASE D
INNER JOIN PRODUCTO_ZONA P
    ON D.PRODUCTO_ID = P.PRODUCTO_ID AND D.ZONA_ID=P.ZONA_ID
ORDER BY ZONA_ID ASC

(291 rows affected)

Total execution time: 00:00:00.118

PEDIDO_ID,PRODUCTO_ID,ZONA_ID,RN
P-0018,Algas,Coruña,1
P-0020,Algas,Coruña,1
P-0029,Algas,Coruña,1
P-0030,Algas,Coruña,1
P-0033,Algas,Coruña,1
P-0042,Algas,Coruña,1
P-0077,Algas,Coruña,1
P-0080,Algas,Coruña,1
P-0081,Algas,Coruña,1
P-0082,Algas,Coruña,1
